## Set Global Seed 

In [1]:
import os
import random
import numpy as np
import torch
import transformers

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed = 260615
set_all_seeds(seed)

print("The global seed " + str(seed))

The global seed 260615


## Hyperparameters

In [2]:
# LANGUAGE

_LANGUAGE_         = 'es'
_DATASET_          = '2019'

In [3]:
# MODEL CLASSIFICATION

_PRETRAINED_LM_    = 'vicgalle/xlm-roberta-large-xnli-anli'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.IA3Config()
_MAX_SEQ_LEN_      = 150

In [4]:
# TRAIN

_OUTPUT_DIR_       = 'checkPointsIA3NLI'
_LOGGING_STEPS_    = 50
_NUM_AUTHORS_      = 512
_K_FOLD_CV_        = 5
_NO_GPUS_          = 1
_BATCH_SIZE_       = int(8 / _NO_GPUS_)
_EPOCHS_           = 10
_LEARNING_RATE_    = 1e-5

# PREDICTIONS

_PRED_DIR_         = 'IA3NLI'

## Other parameters

In [5]:
# LABEL DICTONARIES -----------------------------------------------------------------------

# 2015

age_dict  = {'18-24': 0, '25-34': 1, '35-49': 2, '50-XX': 3}
ageEN_hyp = {0: '18-24', 1: '25-34', 2: '35-49', 3: '50-XX'}
ageES_hyp = {0: 'La edad de esta persona es entre 18 y 24 años', 
             1: 'La edad de esta persona es entre 25 y 34 años', 
             2: 'La edad de esta persona es entre 35 y 49 años', 
             3: 'La edad de esta persona es más de 50 años'}

# 2017

gender_dict    = {'female': 0, 'male':   1}
varietyEN_dict = {'australia': 0, 'canada': 1, 'great britain': 2, 'ireland': 3, 'new zealand': 4, 'united states': 5}
varietyES_dict = {'argentina': 0, 'chile': 1, 'colombia': 2, 'mexico': 3, 'peru': 4, 'spain': 5, 'venezuela': 6}  

genderEN_hyp  = {0: 'I’m a female', 1: 'I’m a male'}
genderES_hyp  = {0: 'Mi nombre es María', 1: 'Mi nombre es José'}

# 2019

bots_dict  = {'human': 0, 'bot': 1}
botsEN_hyp = {0: 'This is a text from a person', 1: 'This is a text from a machine'}
botsES_hyp = {0: 'Humano', 1: 'Bot'}

# 2020 

fakeNews_dict  = {'0': 0, '1': 1}
fakeNewsEN_hyp = {0: 'This author is a normal user', 1: 'This author spreads fake news'}
fakeNewsES_hyp = {0: 'Este autor es un usuario normal', 1: 'Este autor publica noticias falsas'}

# 2021

hateSpeech_dict  = {'0': 0, '1': 1}
hateSpeechEN_hyp = {0: 'This text does not contain hate speech', 1: 'This text expresses prejudice and hate speech'}
hateSpeechES_hyp = {0: 'Este texto es moderado, respetuoso, cortés y civilizado', 1: 'Este texto expresa odio o prejuicios'}

In [6]:
# SET LANGUAGE DICTIONARIES --------------------------------------------------

if _LANGUAGE_ == 'en':
    age_hyp        = ageEN_hyp
    gender_hyp     = genderEN_hyp
    variety_dict   = varietyEN_dict
    fakeNews_hyp   = fakeNewsEN_hyp
    hateSpeech_hyp = hateSpeechEN_hyp
    bots_hyp       = botsEN_hyp 

elif _LANGUAGE_ == 'es':
    age_hyp        = ageES_hyp
    gender_hyp     = genderES_hyp
    variety_dict   = varietyES_dict
    fakeNews_hyp   = fakeNewsES_hyp
    hateSpeech_hyp = hateSpeechES_hyp
    bots_hyp       = botsES_hyp
    
    
# SET LANGUAGE AND DATASET PARAMETERS ----------------------------------------
    
if   _DATASET_ == '2015':
    label_idx  = 2
    class_dict = age_dict
    label_name = 'age'
    label_hyp  = age_hyp
    
elif _DATASET_ == '2017':
    label_idx  = 1
    class_dict = gender_dict
    label_name = 'gender'
    label_hyp  = gender_hyp
    
elif _DATASET_ == '2019':
    label_idx  = 1
    class_dict = bots_dict
    label_name = 'bots'
    label_hyp  = bots_hyp
    
elif _DATASET_ == '2020':
    label_idx  = 1
    class_dict = fakeNews_dict
    label_name = 'fakeNews'
    label_hyp  = fakeNews_hyp
    
elif _DATASET_ == '2021':
    label_idx  = 1
    class_dict = hateSpeech_dict
    label_name = 'hateSpeech'
    label_hyp  = hateSpeech_hyp
    

In [7]:
from transformers import AutoTokenizer, PretrainedConfig

tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
vocab = tokenizer.get_vocab()

config             = PretrainedConfig.from_pretrained(_PRETRAINED_LM_)
nli_label2id       = config.label2id
is_encoder_decoder = config.is_encoder_decoder

You are using a model of type xlm-roberta to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


## Datasets

In [8]:
# GET AUTHORS AND LABELS -----------------------------------------------------

from tools.DataLoaders import BasePAN

baseTrain  = BasePAN(Dir        = 'data/' + _DATASET_,
                     split      = 'train',
                     language   = _LANGUAGE_,
                     label_idx  = label_idx,
                     class_dict = class_dict,
                     label_name = label_name)

baseTest   = BasePAN(Dir        = 'data/' + _DATASET_,
                     split      = 'test',
                     language   = _LANGUAGE_,
                     label_idx  = label_idx,
                     class_dict = class_dict,
                     label_name = label_name)

In [9]:
# GET K-FOLD SPLITS -----------------------------------------------------

crossVal_splits = baseTrain.cross_val(_K_FOLD_CV_, _NUM_AUTHORS_)

crossVal_splits[0][0]

['c270cc9a3be185547804c10cc2985e3c',
 'ef8a2d99b01c8a024172c202951e5893',
 'abf5cbfa07bf8973f3a2d8d3052a03e8',
 'fb29e93c9bdb0dc827f3d2d5751fad2a',
 '52a11f8880a98760047362d065d08799',
 '21fd2a8aa5eb8e49c3b81c2382f3d6b6',
 'd6ffa20cdf6a6d6185b5a8bc26e8c93d',
 'bc2c800ebafce8e75690252a53c4cca7',
 '6631827a81978052c88294ee9be6afe5',
 'e2296ddebe34fbc26966f638739abbf0',
 '5770c9aa6bf073e0674a679b1f5ee88b',
 '62586920282fe7e5df630a439488029c',
 '335cc0daaf8dbb3772b03e077a198d08',
 '9e5a10b5c50af5da00417026ad8ad027',
 '5d17b889e9c842795782f0dee18ca2ed',
 '475b7d7073d9f78fd6fdb7d96692316b',
 '3ce0b034475e1fdc4486b0d7e2faec17',
 '9b84e175729d9f4f25cf53de10183f65',
 '8a50d1f5adc80cc2178f18613d3adf21',
 '9bb3ecf9724c0c85a77281f9822a46f3',
 'f8627e5420297aee1fcea32b62e40a35',
 'f605a79453c32ba2d99bfd361d39deed',
 'd7f0fe2d08596dccfb259474a601e66c',
 'cec0c0d6ed8bb614563935d3a0a74df3',
 'ad41196384738c373f5f2f5332a1ee61',
 'a586df8faf679f44947cd9b91ff1d28',
 '2ffda3b2e84a6036d38be5168b3e0360',
 '

In [10]:
# GET TWEETS -----------------------------------------------------

baseTrain.get_all_data(_TWEET_BATCH_SIZE_, tokenizer, _MAX_SEQ_LEN_, _PREPROCESS_TEXT_, NLI=True, label_hyp=label_hyp, nli_label2id=nli_label2id)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 120000



## Training and Testing

In [11]:
from tools.DataLoaders import DatasetPANnli

baseTest.get_all_data(_TWEET_BATCH_SIZE_, tokenizer, _MAX_SEQ_LEN_, _PREPROCESS_TEXT_, NLI=True, label_hyp=label_hyp, nli_label2id=nli_label2id)

Test = DatasetPANnli(baseTest)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 72000



In [12]:
from transformers import TrainingArguments

samples = 4 * _NUM_AUTHORS_ * int(100 / _TWEET_BATCH_SIZE_)
_LOGGING_STEPS_ = int(samples / _BATCH_SIZE_)

training_args = TrainingArguments(
    learning_rate               = _LEARNING_RATE_,
    num_train_epochs            = _EPOCHS_,
    per_device_train_batch_size = _BATCH_SIZE_,
    per_device_eval_batch_size  = 200,
    logging_steps               = _LOGGING_STEPS_,
    output_dir                  = _OUTPUT_DIR_,
    save_total_limit            = 5,
    overwrite_output_dir        = True,
    remove_unused_columns       = False,
)

In [ ]:
from transformers import AutoAdapterModel, AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossValnli
from transformers import AdapterTrainer
from tools.Testing import compute_author_predictions_nli, compute_author_predictions_nli_LR
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = label_name

f1s_soft = []
f1s_hard = []

f1s_soft_LR = []
f1s_hard_LR = []

for split in range( _K_FOLD_CV_ ):
    
    # loaders for current split ------------------------------------------
    
    authors_train, authors_val = crossVal_splits[split]
    
    Train = DatasetCrossValnli(baseTrain, authors_train)
    Val   = DatasetCrossValnli(baseTrain, authors_val)
    
    
    # initialize model ---------------------------------------------------
    

    static_head_model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_)
    static_head_model.add_adapter(task, config = _ADAPTER_CONFIG_) 
    static_head_model.save_adapter('aux_adapter_'+_PRED_DIR_, task)

    model = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)
    model.load_adapter('aux_adapter_'+_PRED_DIR_, load_as=task)
    #model.load_adapter('pretrained_adapter_NLI', load_as=task)

    model.set_active_adapters(task)
    model.train_adapter(task)


    # create trainer and train -------------------------------------------

    trainer = AdapterTrainer(
        model           = model,
        args            = training_args,
        train_dataset   = Train,
    )
    trainer.args._n_gpu = _NO_GPUS_

    trainer.train()
    
    
    # get predictions ----------------------------------------------------
    
    
    ignore_keys = None
    if is_encoder_decoder:
        ignore_keys = ['encoder_last_hidden_state']

    results            = trainer.predict(Test , ignore_keys = ignore_keys)
    author_predictions = compute_author_predictions_nli(baseTest, results.predictions, task, len(class_dict), nli_label2id)

    # report metrics 

    report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
               'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

    f1s_soft.append( f1_score(author_predictions['true'], author_predictions['pred_soft'], average = 'macro') )
    f1s_hard.append( f1_score(author_predictions['true'], author_predictions['pred_hard'], average = 'macro') )
    
    print("Results with split " + str(split + 1) + ":\n")
    print("soft voting:\n", report['soft'], '\n')
    print("hard voting:\n", report['hard'])


    # get predictions with Logistic Regression----------------------------

    resultsTrain = trainer.predict(Train, ignore_keys = ignore_keys)
    author_predictions_LR = compute_author_predictions_nli_LR(Train, baseTest, resultsTrain.predictions, results.predictions, task, len(class_dict))
    
    f1s_soft_LR.append( f1_score(author_predictions_LR['true'], author_predictions_LR['pred_soft'], average = 'macro') )
    f1s_hard_LR.append( f1_score(author_predictions_LR['true'], author_predictions_LR['pred_hard'], average = 'macro') )
    
    # report metrics 

    report_LR = {'soft': classification_report(author_predictions_LR['true'], author_predictions_LR['pred_soft'], digits=4), 
               'hard': classification_report(author_predictions_LR['true'], author_predictions_LR['pred_hard'], digits=4)}

    print("Results with split " + str(split + 1) + " using LOGISTIC REGRESSION:\n")
    print("soft voting:\n", report_LR['soft'], '\n')
    print("hard voting:\n", report_LR['hard'])

     
    
    # save predictions ----------------------------------------------------
    
    DIR = 'results/' + _DATASET_ + '/' + _LANGUAGE_ + '/' + _PRED_DIR_ + '/' + str(_NUM_AUTHORS_) + '_authors/test_split_' + str(split + 1) + '/'
    if not os.path.exists(DIR):
        os.makedirs(DIR)

    with open(DIR + 'predictions.pickle', 'wb') as f:
        pickle.dump(author_predictions, f)
    
    with open(DIR + 'predictions_LR.pickle', 'wb') as f:
        pickle.dump(author_predictions_LR, f)

    with open(DIR + 'report.txt', 'w') as f:
        f.write("soft voting:\n" + report['soft'] + '\n\n')
        f.write("hard voting:\n" + report['hard'])
        
    with open(DIR + 'report_LR.txt', 'w') as f:
        f.write("soft voting:\n" + report_LR['soft'] + '\n\n')
        f.write("hard voting:\n" + report_LR['hard'])
    


In [16]:
# report statistics

print('Soft results: ', f1s_soft)
print('\nHard results: ', f1s_hard)

f1s_soft = np.array(f1s_soft)
f1s_hard = np.array(f1s_hard)

FewShot_Results = {'soft': [f1s_soft.mean(), f1s_soft.std()], 'hard': [f1s_hard.mean(), f1s_hard.std()]}

print('\n\nSoft statistics: ')
print('\t[avg, std]:', FewShot_Results['soft'])

print('\nHard statistics: ')
print('\t[avg, std]:', FewShot_Results['hard'])

Soft results:  [0.87772464 0.87874201 0.87538925 0.87311028 0.88056244]

Hard results:  [0.87085156 0.87190002 0.87020979 0.86672474 0.87427167]


Soft statistics: 
	[avg, std]: [0.8771057226587893, 0.0026045799555797105]

Hard statistics: 
	[avg, std]: [0.8707915539672667, 0.002458492195639212]


In [17]:
# report statistics

print('Soft results: ', f1s_soft_LR)
print('\nHard results: ', f1s_hard_LR)

f1s_soft_LR = np.array(f1s_soft_LR)
f1s_hard_LR = np.array(f1s_hard_LR)

FewShot_Results_LR = {'soft': [f1s_soft_LR.mean(), f1s_soft_LR.std()], 'hard': [f1s_hard_LR.mean(), f1s_hard_LR.std()]}

print('\n\nSoft statistics with LOGISTIC REGRESSION: ')
print('\t[avg, std]:', FewShot_Results_LR['soft'])

print('\nHard statistics with LOGISTIC REGRESSION: ')
print('\t[avg, std]:', FewShot_Results_LR['hard'])

Soft results:  [0.89387942 0.89329065 0.88996944 0.89108476 0.89276821]

Hard results:  [0.88545996 0.89092676 0.88429817 0.88428888 0.8921102 ]


Soft statistics with LOGISTIC REGRESSION: 
	[avg, std]: [0.8921984956043228, 0.0014527228167052842]

Hard statistics with LOGISTIC REGRESSION: 
	[avg, std]: [0.8874167966109849, 0.0033966693523083944]


## Training with all

In [11]:
from tools.DataLoaders import DatasetPANnli

baseTest.get_all_data(_TWEET_BATCH_SIZE_, tokenizer, _MAX_SEQ_LEN_, _PREPROCESS_TEXT_, NLI=True, label_hyp=label_hyp, nli_label2id=nli_label2id)

Test  = DatasetPANnli(baseTest)
Train = DatasetPANnli(baseTrain)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 96000



In [12]:
from transformers import TrainingArguments

samples = 4 * _NUM_AUTHORS_ * int(100 / _TWEET_BATCH_SIZE_)
_LOGGING_STEPS_ = int(samples / _BATCH_SIZE_)

training_args = TrainingArguments(
    learning_rate               = _LEARNING_RATE_,
    num_train_epochs            = _EPOCHS_,
    per_device_train_batch_size = _BATCH_SIZE_,
    per_device_eval_batch_size  = 200,
    logging_steps               = _LOGGING_STEPS_,
    output_dir                  = _OUTPUT_DIR_,
    save_total_limit            = 5,
    overwrite_output_dir        = True,
    remove_unused_columns       = False,
)

In [ ]:
from transformers import AutoAdapterModel, AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossValnli
from transformers import AdapterTrainer
from tools.Testing import compute_author_predictions_nli, compute_author_predictions_nli_LR
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = label_name

# initialize model ---------------------------------------------------

static_head_model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_)
static_head_model.add_adapter(task, config = _ADAPTER_CONFIG_) 
static_head_model.save_adapter('aux_adapter_'+_PRED_DIR_, task)

model = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)
model.load_adapter('aux_adapter_'+_PRED_DIR_, load_as=task)
#model.load_adapter('pretrained_adapter_NLI', load_as=task)

model.set_active_adapters(task)
model.train_adapter(task)


# create trainer and train -------------------------------------------

trainer = AdapterTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = Train,
)
trainer.args._n_gpu = _NO_GPUS_

trainer.train()


# get predictions ----------------------------------------------------


ignore_keys = None
if is_encoder_decoder:
    ignore_keys = ['encoder_last_hidden_state']

results            = trainer.predict(Test , ignore_keys = ignore_keys)
author_predictions = compute_author_predictions_nli(baseTest, results.predictions, task, len(class_dict), nli_label2id)

# report metrics 

report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
           'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

print("Results:\n")
print("soft voting:\n", report['soft'], '\n')
print("hard voting:\n", report['hard'])


# get predictions with Logistic Regression----------------------------


resultsTrain = trainer.predict(Train, ignore_keys = ignore_keys)
author_predictions_LR = compute_author_predictions_nli_LR(baseTrain, baseTest, resultsTrain.predictions, results.predictions, task, len(class_dict))

# report metrics 

report_LR = {'soft': classification_report(author_predictions_LR['true'], author_predictions_LR['pred_soft'], digits=4), 
           'hard': classification_report(author_predictions_LR['true'], author_predictions_LR['pred_hard'], digits=4)}

print("Results using LOGISTIC REGRESSION:\n")
print("soft voting:\n", report_LR['soft'], '\n')
print("hard voting:\n", report_LR['hard'])



Some weights of the model checkpoint at ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing BartAdapterModel: ['classification_head.dense.weight', 'classification_head.out_proj.weight', 'classification_head.dense.bias', 'classification_head.out_proj.bias']
- This IS expected if you are initializing BartAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torc

Step,Training Loss
5120,0.744400
10240,0.644100


Saving model checkpoint to checkPointsIA3NLI/checkpoint-500
Configuration saved in checkPointsIA3NLI/checkpoint-500/gender/adapter_config.json
Module weights saved in checkPointsIA3NLI/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in checkPointsIA3NLI/checkpoint-500/gender/head_config.json
Module weights saved in checkPointsIA3NLI/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in checkPointsIA3NLI/checkpoint-500/gender/head_config.json
Module weights saved in checkPointsIA3NLI/checkpoint-500/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsIA3NLI/checkpoint-178000] due to args.save_total_limit
Saving model checkpoint to checkPointsIA3NLI/checkpoint-1000
Configuration saved in checkPointsIA3NLI/checkpoint-1000/gender/adapter_config.json
Module weights saved in checkPointsIA3NLI/checkpoint-1000/gender/pytorch_adapter.bin
Configuration saved in checkPointsIA3NLI/checkpoint-1000/gender/head_config.json
Module weights saved in checkPoints

Module weights saved in checkPointsIA3NLI/checkpoint-6500/gender/pytorch_model_head.bin
Configuration saved in checkPointsIA3NLI/checkpoint-6500/gender/head_config.json
Module weights saved in checkPointsIA3NLI/checkpoint-6500/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsIA3NLI/checkpoint-4000] due to args.save_total_limit
Saving model checkpoint to checkPointsIA3NLI/checkpoint-7000
Configuration saved in checkPointsIA3NLI/checkpoint-7000/gender/adapter_config.json
Module weights saved in checkPointsIA3NLI/checkpoint-7000/gender/pytorch_adapter.bin
Configuration saved in checkPointsIA3NLI/checkpoint-7000/gender/head_config.json
Module weights saved in checkPointsIA3NLI/checkpoint-7000/gender/pytorch_model_head.bin
Configuration saved in checkPointsIA3NLI/checkpoint-7000/gender/head_config.json
Module weights saved in checkPointsIA3NLI/checkpoint-7000/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsIA3NLI/checkpoint-4500] due to args.save_total

In [17]:

print("Results:\n")
print("soft voting:\n", report['soft'], '\n')
print("hard voting:\n", report['hard'])

Results:

soft voting:
               precision    recall  f1-score   support

           0     0.7790    0.8283    0.8029      1200
           1     0.8167    0.7650    0.7900      1200

    accuracy                         0.7967      2400
   macro avg     0.7979    0.7967    0.7965      2400
weighted avg     0.7979    0.7967    0.7965      2400
 

hard voting:
               precision    recall  f1-score   support

           0     0.7693    0.8367    0.8016      1200
           1     0.8210    0.7492    0.7834      1200

    accuracy                         0.7929      2400
   macro avg     0.7952    0.7929    0.7925      2400
weighted avg     0.7952    0.7929    0.7925      2400



In [18]:

print("Results using LOGISTIC REGRESSION:\n")
print("soft voting:\n", report_LR['soft'], '\n')
print("hard voting:\n", report_LR['hard'])


Results using LOGISTIC REGRESSION:

soft voting:
               precision    recall  f1-score   support

           0     0.7939    0.8025    0.7982      1200
           1     0.8003    0.7917    0.7960      1200

    accuracy                         0.7971      2400
   macro avg     0.7971    0.7971    0.7971      2400
weighted avg     0.7971    0.7971    0.7971      2400
 

hard voting:
               precision    recall  f1-score   support

           0     0.7779    0.8200    0.7984      1200
           1     0.8097    0.7658    0.7872      1200

    accuracy                         0.7929      2400
   macro avg     0.7938    0.7929    0.7928      2400
weighted avg     0.7938    0.7929    0.7928      2400



In [ ]:
model.save_adapter('Pretrained_Adapters_NLI/'+_DATASET_+'_'+_PRED_DIR_, task)